Do NOT execute the following unless absolutely necessary

In [ ]:
%%time
!wget -c http://ckip.iis.sinica.edu.tw/data/ckiptagger/data.zip

Mount Google Drive (for persistent data storage)

In [1]:
# Check TensorFlow version to make sure it's 1.5.x (or else CKIPTagger won't run)
!pip show tensorflow

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Name: tensorflow
Version: 1.15.2
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /srv/conda/envs/notebook/lib/python3.7/site-packages
Requires: absl-py, tensorboard, protobuf, grpcio, keras-preprocessing, numpy, wheel, astor, gast, termcolor, wrapt, tensorflow-estimator, keras-applications, google-pasta, opt-einsum, six
Required-by: 


Install CKIP neural network-based word segmentation and POS-tagger modules

In [ ]:
!python -m pip install -U ckiptagger

Now upnload your sample traditional Chinese text file

Import WS and POS models

In [1]:
from ckiptagger import data_utils, construct_dictionary, WS, POS

Load word segmentation (WS) model and/or POS-tagging model

In [2]:
%%time
ws = WS("./ckipnn_data")
pos = POS("./ckipnn_data")

CPU times: user 12 s, sys: 4.82 s, total: 16.8 s
Wall time: 18.6 s


In [3]:
%%time

infile          = 'ROClaws20180424C.txt'
# segmented words, no POS tags
outfile_seg     = 'ROClaws20180424C.seg.txt'
# POS tags only, no words
outfile_pos     = 'ROClaws20180424C.pos.txt'
# segmented words + POS tags
outfile_seg_pos = 'ROClaws20180424C.seg.pos.txt'

delim = ' '
BATCH_SIZE = 100
cnt = 0
batch = 0

with open(outfile_seg, "w", encoding='utf8', newline='\n') as fo1:
    with open(outfile_pos, "w", encoding='utf8', newline='\n') as fo2:
        with open(infile, 'r', encoding='utf8') as fi:
            sentences = []
            for line in fi:
                cnt += 1
                batch += 1
                sentences.append(line.strip())
                if batch == BATCH_SIZE:
                    results1 = ws(sentences)
                    for sent in results1:
                        fo1.write(delim.join(sent)+"\n")
                    results2 = pos(results1)
                    for sent in results2:
                        fo2.write(delim.join(sent)+"\n")
                    batch = 0 # reset counter
                    sentences = []
                if cnt % (BATCH_SIZE * 1) == 0:
                    print(f"{cnt} lines processed...")
                    fo1.flush()
                    fo2.flush()
            if len(sentences) > 0:
                results1 = ws(sentences)
                for sent in results1:
                    fo1.write(delim.join(sent)+"\n")
                results2 = pos(results1)
                for sent in results2:
                    fo2.write(delim.join(sent)+"\n")   

delim2 = '_'
with open(outfile_seg_pos, "w", encoding='utf8', newline='\n') as fo3:
    with open(outfile_seg, "r", encoding='utf8') as fi1:
        with open(outfile_pos, "r", encoding='utf8') as fi2:
            for line1 in fi1:
                line2 = next(fi2)
                pairs = zip(line1.strip().split(delim), line2.strip().split(delim))
                fo3.write(delim.join([ f"{p[0]}{delim2}{p[1]}" for p in pairs]) + "\n")


100 lines processed...
200 lines processed...
300 lines processed...
400 lines processed...
500 lines processed...
600 lines processed...
700 lines processed...
800 lines processed...
900 lines processed...
1000 lines processed...
1100 lines processed...
1200 lines processed...
1300 lines processed...
1400 lines processed...
1500 lines processed...
1600 lines processed...
1700 lines processed...
1800 lines processed...
1900 lines processed...
2000 lines processed...
2100 lines processed...
2200 lines processed...
2300 lines processed...
2400 lines processed...
2500 lines processed...
2600 lines processed...
2700 lines processed...
2800 lines processed...
2900 lines processed...
3000 lines processed...
3100 lines processed...
3200 lines processed...
3300 lines processed...
3400 lines processed...
3500 lines processed...
3600 lines processed...
3700 lines processed...
3800 lines processed...
3900 lines processed...
4000 lines processed...
4100 lines processed...
4200 lines processed...
4

Finally, make sure all the files have the same line count

In [ ]:
!wc -l {infile} {outfile_seg} {outfile_pos} {outfile_seg_pos}